# Prima prova di neural network

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
#from tqdm.notebook import tqdm, trange
from tqdm.auto import tqdm, trange
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from skimage.io import imread
from copy import deepcopy
import matplotlib as plt
from sklearn import preprocessing

prendo i dati, li leggo e li splitto

In [ ]:
train = pd.read_csv("data/train_dataset.csv")
train_img = []
for img_name in tqdm(train['id']):
    image_path = 'data/' + str(img_name) + '.png'
    img = imread(image_path, as_gray=True)
    # normalizing the pixel values
    img /= 255.0
    # converting the type of pixel to float 32
    img = img.astype('float32')
    train_img.append(img)

# converting the list to numpy array
X = np.array(train_img)
y = train['label'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.15)

X_train = X_train.reshape(54000, 1, 28, 28)
X_train  = torch.from_numpy(X_train)

# converting the target into torch format
y_train = y_train.astype(int)
y_train = torch.from_numpy(y_train)

# shape of training data
X_train.shape, y_train.shape

In [ ]:
X_test = X_test.reshape(6400, 1, 28, 28)
X_test  = torch.from_numpy(X_test)

# converting the target into torch format
y_test = y_test.astype(int)
y_test = torch.from_numpy(y_test)

In [ ]:
test = TensorDataset(X_test, y_test)
train = TensorDataset(X_train, y_train)

creo i batch, randomizzando i dati, con un dataloader

In [ ]:
batch_size = 50

# Create data loaders.
test = DataLoader(test, batch_size=batch_size, shuffle=True)
train = DataLoader(train, batch_size=batch_size, shuffle=True)

for X, y in test:
    print(f"Shape of X : {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# creo la neural network

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        # take in input resistance, age, heater_r and heater_V
        self.linear_relu_stack = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4 * 7 * 7, 10)
        )


    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        logits = logits.view(x.size(0), -1)
        logits = self.linear_layers(logits)
        return logits


In [ ]:
model = NeuralNetwork().to(device)
print(model)

In [ ]:
#loss_fn = nn.MSELoss()
loss_2 = nn.MSELoss()
#loss_fn = nn.L1Loss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-7, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=0.07)
loss_fn = nn.CrossEntropyLoss()

In [ ]:


def trainf(dataloader, model, loss_fn, optimizer):
    global best_model
    best_loss = float("inf")
    size = len(dataloader.dataset)
    model.double()
    model.train()
    for batch , (X, Y) in enumerate(dataloader):
        gas_loss = [([],[]) for _ in range(5)]
        X, Y = X.to(device), Y.to(device)
        loss = float("inf")
        optimizer.zero_grad()
        # Compute prediction error
        for x_line, y_line in zip(X, Y):
            pred = model(x_line)
            for i, (gas, y_lin) in enumerate(zip(pred, y_line)):
                gas_loss[i][0].append(y_lin)
                gas_loss[i][1].append(gas)
            loss = loss_fn(pred, y_line)
            loss.backward()
        # Backpropagation
        optimizer.step()

        if loss < best_loss:
            best_loss = loss
            best_model = deepcopy(model)

        if batch % 100 == 0:
            for i, gas in enumerate(gas_loss):
                pass
                #print(f"gas {i}: loss = {loss_2(torch.tensor(gas[0]), torch.tensor(gas[1])).item()}")
            loss, current = loss.item(), batch * len(X)
            train_losses.append(loss)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def testf(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        gas_avg_loss = [0 for _ in range(5)]
        for X, Y in dataloader:
            gas_loss = [([],[]) for _ in range(5)]
            X, Y = X.to(device), Y.to(device)
            for x_line, y_line in zip(X, Y):
                pred = model(x_line)
                for i, (gas, y_lin) in enumerate(zip(pred, y_line)):
                    gas_loss[i][0].append(y_lin)
                    gas_loss[i][1].append(gas)
                test_loss += loss_fn(pred, y_line).item()
                #correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            for i, gas in enumerate(gas_loss):
                for yp, pre in zip(gas[0],gas[1]):
                    gas_avg_loss[i] += loss_fn(torch.tensor(yp), torch.tensor(pre)).item()
    test_loss /= num_batches * batch_size
    for i, num in enumerate(gas_avg_loss):
        print(f'gas {i}: avg loss = {num / (num_batches * batch_size)}')
    #correct /= size
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    val_losses.append(test_loss)
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 20
train_losses = []
val_losses = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    trainf(train, model, loss_fn, optimizer)
    testf(test, model, loss_fn)
print("Done!")

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()